In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import keras
from keras.models import Sequential
from keras.layers import Dense

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/fra

In [2]:
# Download the data and read it into a pandas dataframe
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
# Split data into predictors and target
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

print(predictors,target)

      Cement  Blast Furnace Slag  Fly Ash  Water  Superplasticizer  \
0      540.0                 0.0      0.0  162.0               2.5   
1      540.0                 0.0      0.0  162.0               2.5   
2      332.5               142.5      0.0  228.0               0.0   
3      332.5               142.5      0.0  228.0               0.0   
4      198.6               132.4      0.0  192.0               0.0   
...      ...                 ...      ...    ...               ...   
1025   276.4               116.0     90.3  179.6               8.9   
1026   322.2                 0.0    115.6  196.0              10.4   
1027   148.5               139.4    108.6  192.7               6.1   
1028   159.1               186.7      0.0  175.6              11.3   
1029   260.9               100.5     78.3  200.6               8.6   

      Coarse Aggregate  Fine Aggregate  Age  
0               1040.0           676.0   28  
1               1055.0           676.0   28  
2                932.

In [4]:
# Normalize the data by substracting the mean and dividing by the standard deviation
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [5]:
# Save the number of predictors to n_cols since we will need this number when building our network
n_cols = predictors_norm.shape[1]
n_cols

8

In [14]:
# Randomly split the data into a training and test sets by holding 30% of the data for testing
# For part B, if we want the data to be normalized, we give the function the number 1

def split(normalize):
    if normalize == 1:
        X_train, X_test, y_train, y_test = train_test_split(predictors_norm,target, test_size=0.30, random_state=40)
    else:
        X_train, X_test, y_train, y_test = train_test_split(predictors,target, test_size=0.30, random_state=40)
    
    return X_train, X_test, y_train, y_test
        

In [15]:
# Define the regression model
# For part D, if we want the 3 hidden layers, we give the function the number 1

def regression_model(three_lay):
    # create model
    if three_lay == 1:
        model = Sequential()
        model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(1))
        
    else:
        model = Sequential()
        model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
        model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


In [16]:
# Run the model
# For part C, we can give the function the number of epochs we want the model to go through
# For part D, the same as for the regression_model function

def run_model(normalize, three_lay, epo):
    
    # Split the data
    X_train, X_test, y_train, y_test = split(normalize)
    
    # build the model
    model = regression_model(three_lay)
    
    # fit the model
    model.fit(X_train, y_train, epochs=epo, verbose=0)

    # Evaluate the model on the test data
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return mse


In [18]:
# Create a list of 50 mean squared errors for each stage of the project

list_mseA = []
list_mseB = []
list_mseC = []
list_mseD = []

# Run each model 50 times, for part A B C and D
for i in range(50):
    # For A
    mA = run_model(0,0,50)
    list_mseA.append(mA)
    
    # For B
    mB = run_model(1,0,50)
    list_mseB.append(mB)
    
    # For C
    mC = run_model(1,0,100)
    list_mseC.append(mC)
    
    # For D
    mD = run_model(1,1,100)
    list_mseD.append(mD)
    
    
# Mean and the standard deviation of the mean squared error for each stage

mse_arrayA = np.array(list_mseA)
mean_mseA = np.mean(mse_arrayA)
std_dev_mseA = np.std(mse_arrayA)

mse_arrayB = np.array(list_mseB)
mean_mseB = np.mean(mse_arrayB)
std_dev_mseB = np.std(mse_arrayB)

mse_arrayC = np.array(list_mseC)
mean_mseC = np.mean(mse_arrayC)
std_dev_mseC = np.std(mse_arrayC)

mse_arrayD = np.array(list_mseD)
mean_mseD = np.mean(mse_arrayD)
std_dev_mseD = np.std(mse_arrayD)

print("Mean of Mean Squared Errors A:", mean_mseA)
print("Standard Deviation of Mean Squared Errors A:", std_dev_mseA)

print("Mean of Mean Squared Errors B:", mean_mseB)
print("Standard Deviation of Mean Squared Errors B:", std_dev_mseB)

print("Mean of Mean Squared Errors C:", mean_mseC)
print("Standard Deviation of Mean Squared Errors C:", std_dev_mseC)

print("Mean of Mean Squared Errors D:", mean_mseD)
print("Standard Deviation of Mean Squared Errors D:", std_dev_mseD)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Mean of Mean Squared Errors A: 396.39715986285233
Standard Deviation of Mean Squared Errors A: 495.2817379703225
Mean of Mean Squared Errors B: 385.78578566382976
Standard Deviation of Mean Squared Errors B: 113.05626942103707
Mean of Mean Squared Errors C: 173.78647425061735
Standard Deviation of Mean Squared Errors C: 12.370643981829152
Mean of Mean Squared Errors D: 90.65106362794502
Standard Deviation of Mean Squared Errors D: 22.318462985830276


In [ ]:
'''Mean of Mean Squared Errors A: 396.39715986285233
Standard Deviation of Mean Squared Errors A: 495.2817379703225

Mean of Mean Squared Errors B: 385.78578566382976
Standard Deviation of Mean Squared Errors B: 113.05626942103707

Mean of Mean Squared Errors C: 173.78647425061735
Standard Deviation of Mean Squared Errors C: 12.370643981829152

Mean of Mean Squared Errors D: 90.65106362794502
Standard Deviation of Mean Squared Errors D: 22.318462985830276


The mean of MSE's in A is very large, probably due to the low number of epochs and neurons. The standard deviation of the MSE's is 
even larger, indicating a very big variance in MSE values for each trial, probably due to the lack of normalization of the target
variables.

For stage B, we can confirm that normalizing the data did not change the mean much, since we still have the same number of epochs and
neurons, but it did reduce the standard deviation drastically, since the training environment for the model becomes more stable and 
consistent.

Stage C improves on stage B. The doubling of the number of epochs reduces the mean of the MSE by more than a factor of 2. It also 
massively reduces the standard deviation, since the larger number of epochs gives the model more time to converge.

Stage D also improves on stage B. The higher number of epochs and layers of neurons reduces the mean and the standard deviation. 
It it interesting to note that the standard deviation in C is smaller than in D, probably because the extra layers of neurons add
more variability to the way the model converges.

'''